In [4]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import gc
import tqdm
import numpy as np
import pandas as pd

In [5]:
path = 'Downloads/train_data/'

In [6]:
# для удобства конкатенации датасетов пишем функцию извлечения индекса 

def get_index(file_name):
        match = re.search(r'\d+', file_name)
        return int(match.group(0)) if match else -1

In [7]:
# читаем датасет

def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 12, columns=None, verbose=False) -> pd.DataFrame:
    res = pd.DataFrame()
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                              if filename.startswith('train')])
    print(dataset_paths)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res = chunk if len(res) == 0 else pd.concat([res, chunk], axis=0)

    return res.reset_index(drop=True)

In [8]:
# так как всего 26 млн строк и 3 млн уникальных номеров id соберем один датасет с 3 млн сроками, отсортируем 
# по последнему кредиту (порядковый номер кредита - rn). Удалим лишниее столбцы (статусы платежей и столбцы со 
# значениями "не определно") и закодируем данные

def make_contact(data):

        data = data.drop(columns=['pre_loans_total_overdue'])
        
        column= data.columns[2:].tolist()
        data_code = pd.get_dummies(data[column], columns=column, dtype='int8')
        data_code = pd.concat([data[['id', 'rn']], data_code], axis = 1)
    
        total_loans = data_code.groupby('id')['rn'].max().reset_index()
        total_loans = total_loans.rename(columns={'rn': 'total_loans'})

        features = data_code.groupby('id').sum().reset_index()
        features = features.drop(columns='rn')
        features = features.merge(total_loans, on='id')

        return features

In [9]:
def prepare_transactions_dataset(path_to_dataset: str, num_parts_to_preprocess_at_once: int = 4, num_parts_total: int=50,
                                 save_to_path=None, verbose: bool=False):

    preprocessed_frames = pd.DataFrame()

    for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once,
                                                             verbose=verbose)

        transactions_frame = make_contact(transactions_frame)
  
        preprocessed_frames = transactions_frame if len(preprocessed_frames) == 0 else pd.concat([preprocessed_frames, transactions_frame], axis = 0)
             
        gc.collect()
    return preprocessed_frames.reset_index(drop=True)

In [10]:
data = prepare_transactions_dataset(path, num_parts_to_preprocess_at_once=2, num_parts_total=12,
                                    save_to_path='Downloads/train_data/')

Transforming transactions data:   0%|          | 0/6 [00:00<?, ?it/s]

['Downloads/train_data/train_data_0.pq', 'Downloads/train_data/train_data_1.pq', 'Downloads/train_data/train_data_10.pq', 'Downloads/train_data/train_data_11.pq', 'Downloads/train_data/train_data_2.pq', 'Downloads/train_data/train_data_3.pq', 'Downloads/train_data/train_data_4.pq', 'Downloads/train_data/train_data_5.pq', 'Downloads/train_data/train_data_6.pq', 'Downloads/train_data/train_data_7.pq', 'Downloads/train_data/train_data_8.pq', 'Downloads/train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path Downloads/train_data/train_data_0.pq
chunk_path Downloads/train_data/train_data_1.pq
['Downloads/train_data/train_data_0.pq', 'Downloads/train_data/train_data_1.pq', 'Downloads/train_data/train_data_10.pq', 'Downloads/train_data/train_data_11.pq', 'Downloads/train_data/train_data_2.pq', 'Downloads/train_data/train_data_3.pq', 'Downloads/train_data/train_data_4.pq', 'Downloads/train_data/train_data_5.pq', 'Downloads/train_data/train_data_6.pq', 'Downloads/train_data/train_data_7.pq', 'Downloads/train_data/train_data_8.pq', 'Downloads/train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

...

chunk_path Downloads/train_data/train_data_4.pq
chunk_path Downloads/train_data/train_data_5.pq
['Downloads/train_data/train_data_0.pq', 'Downloads/train_data/train_data_1.pq', 'Downloads/train_data/train_data_10.pq', 'Downloads/train_data/train_data_11.pq', 'Downloads/train_data/train_data_2.pq', 'Downloads/train_data/train_data_3.pq', 'Downloads/train_data/train_data_4.pq', 'Downloads/train_data/train_data_5.pq', 'Downloads/train_data/train_data_6.pq', 'Downloads/train_data/train_data_7.pq', 'Downloads/train_data/train_data_8.pq', 'Downloads/train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path Downloads/train_data/train_data_6.pq
chunk_path Downloads/train_data/train_data_7.pq
['Downloads/train_data/train_data_0.pq', 'Downloads/train_data/train_data_1.pq', 'Downloads/train_data/train_data_10.pq', 'Downloads/train_data/train_data_11.pq', 'Downloads/train_data/train_data_2.pq', 'Downloads/train_data/train_data_3.pq', 'Downloads/train_data/train_data_4.pq', 'Downloads/train_data/train_data_5.pq', 'Downloads/train_data/train_data_6.pq', 'Downloads/train_data/train_data_7.pq', 'Downloads/train_data/train_data_8.pq', 'Downloads/train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path Downloads/train_data/train_data_8.pq
chunk_path Downloads/train_data/train_data_9.pq


In [11]:
data.head(15) # смотрим, как выглядит датасет

,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,...,pre_loans90_10,enc_loans_credit_type_6,enc_loans_credit_type_7,pre_loans_max_overdue_sum_0,pre_loans3060_0,pre_loans3060_3,pre_loans5_10,pre_loans530_17,pre_loans3060_4,pre_loans6090_0
0,0,0,1,1,1,1,2,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,1,0,0,0,0,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,3,1,0,2,1,3,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1,0,0,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,3,0,0,1,1,0,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,0,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data.shape # смотрим размер датасета

(3000000, 419)

In [13]:
data.fillna(0, inplace=True) # заполняем пропуски

In [14]:
data

,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,...,pre_loans90_10,enc_loans_credit_type_6,enc_loans_credit_type_7,pre_loans_max_overdue_sum_0,pre_loans3060_0,pre_loans3060_3,pre_loans5_10,pre_loans530_17,pre_loans3060_4,pre_loans6090_0
0,0,0,1,1,1,1,2,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,1,0,0,0,0,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,3,1,0,2,1,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2499995,1,1,0,0,0,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2999996,2499996,0,3,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2999997,2499997,0,1,0,1,0,1,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2999998,2499998,0,0,0,1,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
data.duplicated() # проверяем на наличие дубликатов

0          False
1          False
2          False
3          False
4          False
           ...  
2999995    False
2999996    False
2999997    False
2999998    False
2999999    False
Length: 3000000, dtype: bool

In [16]:
df_target = pd.read_csv('train_target.csv') # загружаем целевые значения 

In [17]:
df_target.head() # смотрим датасет

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [18]:
df_target.flag.unique()

array([0, 1])

In [19]:
df_target.flag.value_counts()

flag
0    2893558
1     106442
Name: count, dtype: int64

# Вывод по предварительному анализу

У таргета 3 млн строк несбалансированной выборки, надо их уровнять

In [20]:
# объеденяем фичи и целевую переменую 

df_with_target = data.merge(df_target, on ='id')

In [21]:
# запишем фрейм в файл

df_with_target.to_csv('ML_final_preprocessing.csv', index = False)